In [2]:
# obtain the 1,000 most frequent words
!cat normalized_titles.txt | tr " " "\n" | grep "...." | sort | uniq -c | sort -nr | head -1000 | grep -oE '[^ ]+$' > top_words.txt

sort: Broken pipe


In [3]:
def get_neighbours(model, query, threshold=0.1):
    results = model.get_nearest_neighbors(query)
    neighbours = []
    for result in results:
        if result[0] >= threshold:
            neighbours.append(result[1])
    return neighbours

In [15]:
import fasttext
model_epochs25 = fasttext.load_model("epochs25_title_model.bin")

In [16]:
model_epochs25.get_nearest_neighbors('iphone')

[(0.8673823475837708, '4s'),
 (0.8487353920936584, 'apple'),
 (0.8047859072685242, 'ipod'),
 (0.7243606448173523, 'ipad'),
 (0.6965983510017395, '3gs'),
 (0.6798641085624695, 'fabshell'),
 (0.6634081602096558, '4th'),
 (0.6347259879112244, 'nauticase'),
 (0.6113036870956421, 'slimshell'),
 (0.6056530475616455, 'candyshell')]

In [17]:
filepath = 'top_words.txt'
op_file = open('synonyms.csv','w')

with open(filepath) as fp:
   line = fp.readline()
   while line:
       line = line.strip()
       synonyms = get_neighbours(model_epochs25, line, 0.65)
       synonym_entry = f"{line},{','.join(synonyms)}\n"
       op_file.write(synonym_entry)
       line = fp.readline()

op_file.close()

In [18]:
# Since OpenSearch is running in a Docker container, we need to copy the synonyms.csv file to the container. This directory is mounted in docker-compose
!cp synonyms.csv /Users/sengopal/build/my-git/search_with_machine_learning_course/docker/config/